In [12]:
import os
import pandas as pd
import io
import unicodedata


In [21]:
root_files = os.listdir('coral_and_sponges_all/data/pub/data/paleo/coral/')
#root_files = root_files[0:3] + root_files[4:6] + root_files[9:12]
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'coral_and_sponges_all/data/pub/data/paleo/coral/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'coral_and_sponges_all/data/pub/data/paleo/coral/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [22]:
len(sub_files)

195

In [25]:
def create_coral_file_df(filename):
    try:
        coral_file = open(filename,'r')
        file_in = coral_file.readlines()

        for i in range(len(file_in)):
            line = file_in[i]
            if "Location:" in line:
                location = line.split(": ")[1]
            if "Northernmost_Latitude:" in line:
                northernmost_lat = line.split(": ")[1]
            if "Southernmost_Latitude:" in line:
                southernmost_lat = line.split(": ")[1]
            if "Easternmost_Longitude:" in line:
                easternmost_lat = line.split(": ")[1]
            if "Westernmost_Longitude:" in line:
                westernmost_lat = line.split(": ")[1]
            if "Elevation:" in line:
                elevation = line.split(": ")[1].split(' ')[0]
            if "Earliest_Year:" in line:
                earliest_year = line.split(": ")[1]
            if "Most_Recent_Year:" in line:
                most_recent_year = line.split(": ")[1]
            if "Species_Name:" in line:
                species = line.split(": ")[1].strip('\n')
            if not line.startswith('#'):
                print(line)
                age_line = i
                extracted = ' '.join(file_in[age_line:])
                # print(extracted)

        try:
            lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
            lon = (float(easternmost_lat) + float(westernmost_lat)) / 2

            data = io.StringIO(extracted)
            df = pd.read_csv(data, sep="\t", error_bad_lines=False)
            df = df[['age', 'd18O']]
            df = df[df['d18O'] != 'NAN']
            df['lat'] = lat
            df['lon'] = lon
            df['elevation'] = elevation
            df['species'] = species
            return(df)

        except Exception as e:
            print("ERROR : "+str(e))
            # print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
            # print('hi')
    except Exception as e:
            print("ERROR : "+str(e))

Quick Proof of Concept for the function above:

In [26]:
all_coral_df.to_csv('all_coral.csv')


In [27]:
filename = 'coral_and_sponges_all/atlantic/barbados_coral18o.txt'
#"coral_and_sponges_all/atlantic/goodkin2008-sst_noaa.txt"
#'swart1998-st-1_noaa.txt'

create_coral_file_df(filename)

ERROR : [Errno 2] No such file or directory: 'coral_and_sponges_all/atlantic/barbados_coral18o.txt'


In [ ]:
all_coral_df = pd.DataFrame(columns=['age', 'd18O', 'lat', 'lon', 'elevation', 'species'])
for file in sub_files:
    print(file)
    coral_df = create_coral_file_df(file)
    all_coral_df = pd.concat([all_coral_df, coral_df])

all_coral_df.reset_index
all_coral_df


coral_and_sponges_all/data/pub/data/paleo/coral/atlantic/moses2006-side_noaa.txt
﻿﻿# Buccoo Reef - Stable Isotope and Luminescence Data

age	d18O	d13C

1990	-4.1650	-2.2900

1989	-4.3400	-2.0975

1988	-4.1175	-1.9900

1987	-4.2375	-2.0325

1986	-4.4175	-1.7675

1985	-4.7750	-2.9525

1984	-4.2025	-2.8050

1983	-3.6625	-1.0375

1982	-3.6400	-0.8900

1981	-3.9050	-1.1650

1980	-4.3825	-2.2375

1979	-4.3225	-2.2750

1978	-4.6725	-2.1750

1977	-4.0400	-1.9225

1976	-4.3225	-1.6200

1975	-4.7375	-2.7350

1974	-4.6250	-2.8250

1973	-4.2700	-1.6575

1972	-4.0400	-1.9475

1971	-3.7525	-1.2450

1970	-4.1700	-1.5400

1969	-4.5500	-2.2625

1968	-3.9075	-1.0500

1967	-4.1575	-2.1225

1966	-3.7625	-1.0100

1965	-4.3925	-2.1425

1964	-3.9125	-1.1850

1963	-3.6725	-0.9950

1962	-4.0225	-1.8100

1961	-4.1525	-1.7175

1960	-3.7775	-0.8425

1959	-3.4000	-0.4400

1958	-3.8875	-1.0900

1957	-4.1400	-1.8125

1956	-4.3525	-2.2525

1955	-4.2150	-1.9600

1954	-4.1150	-1.9725

1953	-4.0425	-1.6175

1952	-4.1100


1958.87	25.69	-3.4

1958.95	24.02	-3.19

1959.04	22.85	-3.35

1959.12	22.44	-2.85

1959.2	21.81	-3

1959.29	22.09	-2.94

1959.37	22.42	-3.01

1959.45	23.07	-3.09

1959.54	24.58	-3.14

1959.62	25.67	-3.09

1959.7	26.48	-3.32

1959.79	26.25	-3.02

1959.87	25.22	-2.99

1959.95	23.35	-2.89

1960.04	22.44	-2.69

1960.12	22.2	-2.56

1960.2	22.34	-2.51

1960.29	22.6	-2.65

1960.37	22.94	-2.75

1960.45	23.79	-2.86

1960.54	24.85	-3.08

1960.62	25.92	-3.31

1960.7	26.46	-3.52

1960.79	26.38	-3.39

1960.87	25.4	-3.47

1960.95	23.71	-3.41

1961.04	22.36	-3.19

1961.12	21.82	-3.07

1961.2	22.24	-3.03

1961.29	22.78	-3.14

1961.37	23.4	-3.25

1961.45	23.84	-3.33

1961.54	24.96	-3.3

1961.62	26.28	-3.47

1961.7	26.65	-3.79

1961.79	26.5	-3.52

1961.87	25.48	-3.59

1961.95	24.56	-3.63

1962.04	23.11	-3.33

1962.12	22.44	-3.24

1962.2	22.84	-3.12

1962.29	22.91	-2.95

1962.37	23.22	-2.92

1962.45	23.79	-3.06

1962.54	24.93	-3.34

1962.62	26.07	-3.21

1962.7	26.73	-3.88

1962.79	26.53	-3.66

1962.87	2

1538.87	-3.7	0.08	NAN

1538.79	-3.66	0.2	NAN

1538.71	-3.64	0.06	NAN

1538.62	-3.48	0.28	NAN

1538.54	-3.32	0.5	NAN

1538.46	-3.2	0.58	NAN

1538.37	-3.21	0.6	NAN

1538.29	-3.23	0.33	NAN

1538.21	-3.13	0.16	NAN

1538.12	-3.58	0.32	NAN

1538.04	-3.76	0.07	NAN

1537.96	-3.92	-0.04	NAN

1537.87	-4.04	-0.14	NAN

1537.79	-3.83	-0.16	NAN

1537.71	-4.13	-0.22	NAN

1537.62	-3.93	-0.32	NAN

1537.54	-3.77	-0.25	NAN

1537.46	-3.24	-0.26	NAN

1537.37	-3.34	-0.05	NAN

1537.29	-3.23	0.12	NAN

1537.21	-2.95	0.36	NAN

1537.12	-3.07	0.37	NAN

1537.04	-3.18	0.51	NAN

1536.96	-3.11	0.45	NAN

1536.87	-3.28	0.35	NAN

1536.79	-3.57	0.45	NAN

1536.71	-3.66	0.4	NAN

1536.62	-3.7	0.29	NAN

1536.54	-3.61	0.21	NAN

1536.46	-3.63	0.17	NAN

1536.37	-3.56	0.1	NAN

1536.29	-3.49	0.04	NAN

1536.21	-3.22	-0.01	NAN

1536.12	-3.26	0.53	NAN

1536.04	-3.31	0.83	NAN

1535.96	-3.5	0.74	NAN

1535.87	-3.63	0.56	NAN

1535.79	-3.84	0.24	NAN

1535.71	-3.75	0.16	NAN

1535.62	-3.66	-0.15	NAN

1535.54	-3.56	0.16	NAN

1535.46	-3.34	0


1691.791	-4.17	8.994299	-0.49

1691.958	-4.61	8.907324	-0.42

1692.125	-4.65	8.907265	-0.98

1692.291	-4.4	9.064534	-1.73

1692.458	-4.11	9.181955	-1.95

1692.625	-3.99	9.206425	-1.57

1692.791	-4.19	9.039533	-0.24

1692.958	-4.84	8.998630	0.09

1693.125	-4.71	8.907343	-0.44

1693.291	-3.97	9.004451	-1.35

1693.458	-4.01	9.174307	-0.37

1693.625	-3.64	9.203133	-0.1

1693.791	-4.07	9.067942	-0.07

1693.958	-4.49	8.926982	-0.04

1694.125	-4.56	8.887782	-0.48

1694.291	-4.63	8.923591	-0.92

1694.458	-4.33	9.117621	-0.56

1694.625	-4.02	9.188443	-0.2

1694.791	-4.43	9.064751	-0.33

1694.958	-4.74	8.939388	-0.4

1695.125	-4.95	8.862461	-0.42

1695.291	-4.56	9.011369	-0.77

1695.458	-4.08	9.114728	-0.79

1695.625	-3.5	9.254390	-0.46

1695.791	-3.98	9.100268	-0.21

1695.958	-4.11	8.999233	-0.01

1696.125	-3.9	8.969295	0.12

1696.291	-3.96	9.112620	-0.77

1696.458	-3.91	9.223977	-1.16

1696.625	-3.75	9.303367	-1.05

1696.791	-4.1	9.171245	-0.81

1696.958	-4.45	9.026438	-0.57

1697.125	-4.51	8

1941.625	-4.14	NAN	-0.81

1941.791	-4.43	NAN	-0.58

1941.958	-4.68	NAN	-0.3

1942.125	-4.98	NAN	-0.38

1942.291	-4.75	NAN	-0.82

1942.458	-4.35	NAN	-1.17

1942.625	-4.1	NAN	-0.94

1942.791	-4.26	NAN	-0.47

1942.958	-4.72	NAN	-0.63

1943.125	-4.89	NAN	-0.61

1943.291	-4.73	NAN	-0.89

1943.458	-4.31	NAN	-0.67

1943.625	-4.2	NAN	-0.41

1943.791	-4.36	NAN	-0.47

1943.958	-4.62	NAN	-0.52

1944.125	-4.82	NAN	-0.44

1944.291	-4.8	NAN	-0.45

1944.458	-4.24	NAN	-0.59

1944.625	-3.84	NAN	-0.82

1944.791	-4.02	NAN	-0.52

1944.958	-4.41	NAN	-0.28

1945.125	-4.63	NAN	-0.4

1945.291	-4.76	NAN	-0.47

1945.458	-4.33	NAN	-0.6

1945.625	-3.73	NAN	-0.57

1945.791	-4.07	NAN	-0.25

1945.958	-4.38	NAN	-0.31

1946.125	-4.79	NAN	-0.1

1946.291	-4.79	NAN	-0.07

1946.458	-4.44	NAN	-0.48

1946.625	-3.81	NAN	-0.65

1946.791	-4.2	NAN	-0.27

1946.958	-4.35	NAN	-0.85

1947.125	-4.86	NAN	-0.41

1947.291	-4.71	NAN	-0.66

1947.458	-4.04	NAN	-0.88

1947.625	-3.9	NAN	-0.71

1947.791	-4.48	NAN	-0.58

1947.958	-4.77	NAN	-0

...


78.55	1953.89	-4.79	8.809	-0.81	4.506

78.41	1953.97	-4.79	8.828	-1.01	4.315

78.28	1954.04	-4.82	8.825	-0.98	4.328

78.15	1954.12	-4.87	8.791	-0.85	4.261

78.01	1954.2	-4.89	8.83	-0.5	4.245

77.88	1954.27	-4.91	8.834	-0.45	4.21

77.75	1954.36	-4.79	8.842	-0.66	4.197

77.61	1954.44	-4.82	8.885	-1	4.037

77.48	1954.51	-4.82	8.86	-1.17	4.187

77.35	1954.59	-4.79	8.916	-0.84	4.101

77.21	1954.84	-4.88	8.912	-0.75	4.133

77.08	1954.9	-5.06	8.894	-0.61	4.202

76.95	1954.94	-5.07	8.87	-0.56	4.281

76.81	1954.99	-5.02	8.861	-0.53	4.405

76.68	1955.04	-5.02	8.796	-0.62	4.312

76.55	1955.09	-5.04	8.79	-0.79	4.316

76.41	1955.13	-5.05	8.785	-0.89	4.276

76.28	1955.18	-5.06	8.782	-0.94	4.415

76.15	1955.25	-5.03	8.798	-0.96	4.48

76.01	1955.31	-5.03	8.796	-1.01	4.439

75.88	1955.38	-4.91	8.773	-1.12	4.468

75.75	1955.45	-4.79	8.833	-1.1	4.365

75.61	1955.52	-4.8	8.886	-1.1	4.216

75.48	1955.58	-4.83	8.885	-1.12	4.371

75.35	1955.65	-4.83	8.878	-1.01	4.415

75.21	1955.71	-4.78	8.897	-0.81	4.354




2005.75	-2.813839613	-6.009567392

2005.67	-2.532934371	-5.912154145

2005.58	-3.006468783	-5.844433284

2005.50	-2.341126561	-5.697225809

2005.42	-2.580990262	-5.898660434

2005.33	-2.252680205	-5.742119388

2005.25	-2.181013132	-5.694551206

2005.17	-2.232562065	-5.562185287

2005.08	-2.513832565	-5.612494703

2005.00	-2.774360317	-5.692324227

2004.92	-2.679366115	-5.888669011

2004.83	-2.388009895	-5.88234634

2004.75	-2.364346067	-5.826717564

2004.67	-2.289524662	-5.848998691

2004.58	-2.402854023	-5.72308354

2004.50	-2.292414136	-5.651582321

2004.42	-2.134325498	-5.710901198

2004.33	-2.49774568	-5.782274833

2004.25	-2.479436797	-5.711155133

2004.17	-2.20408535	-5.653072357

2004.08	-2.054747343	-5.72081995

2004.00	-2.224431276	-5.736641884

2003.92	-2.394921303	-5.700724125

2003.83	-2.588915348	-5.858946323

2003.75	-2.435576916	-5.754748344

2003.67	-2.250307322	-5.822499752

2003.58	-2.266679049	-5.661894798

2003.50	-2.341973543	-5.711872578

2003.42	-2.548348188	-5.


1890.58	-1.18198895	-5.588158235

1890.50	-1.323447727	-5.642721868

1890.42	-1.548655458	-5.69200609

1890.33	-1.519241738	-5.477691772

1890.25	-1.44061333	-5.58157009

1890.17	-1.316447616	-5.411071777

1890.08	-1.386606398	-5.59844259

1890.00	-1.372280051	-5.441188276

1889.92	-1.053575189	-5.444098431

1889.83	-1.040335058	-5.535820082

1889.75	-1.324772461	-5.494286747

1889.67	-1.120168567	-5.439157486

1889.58	-1.126681524	-5.410509877

1889.50	-1.040890377	-5.354606602

1889.42	-1.195760034	-5.37309196

1889.33	-1.430441009	-5.440114841

1889.25	-1.385475625	-5.435017989

1889.17	-1.401785016	-5.326981544

1889.08	-1.630599022	-5.352311611

1889.00	-1.938080072	-5.468225002

1888.92	-1.940883517	-5.548604965

1888.83	-1.360410333	-5.365474701

1888.75	-1.495366335	-5.521461964

1888.67	-1.486262918	-5.602851391

1888.58	-1.4771595	-5.684240818

1888.50	-1.555885196	-5.552816391

1888.42	-1.260182261	-5.510178566

1888.33	-1.067151785	-5.390872478

1888.25	-1.281351566	-5.503


1867.42	-5.1845	-5.084	-0.101

1867.5	-5.2125	-5.095	-0.118

1867.58	-5.2139	-5.128	-0.086

1867.67	-5.2967	-5.158	-0.139

1867.75	-5.3078	-5.222	-0.086

1867.83	-5.3233	-5.279	-0.044

1867.92	-5.2037	-5.201	-0.003

1868	-4.9736	-5.135	0.161

1868.08	-4.8262	-5.095	0.269

1868.17	-4.7972	-5.074	0.277

1868.25	-4.7956	-5.081	0.285

1868.33	-4.7347	-5.081	0.346

1868.42	-4.9439	-5.084	0.14

1868.5	-4.9085	-5.095	0.186

1868.58	-4.8803	-5.128	0.248

1868.67	-4.8663	-5.158	0.292

1868.75	-4.9669	-5.222	0.255

1868.83	-5.0877	-5.279	0.191

1868.92	-4.8621	-5.201	0.339

1869	-4.7225	-5.135	0.413

1869.08	-4.7525	-5.095	0.343

1869.17	-4.8938	-5.074	0.18

1869.25	-4.9963	-5.081	0.085

1869.33	-4.9842	-5.081	0.097

1869.42	-5.0411	-5.084	0.043

1869.5	-5.1072	-5.095	-0.012

1869.58	-5.2326	-5.128	-0.105

1869.67	-5.3255	-5.158	-0.168

1869.75	-5.402	-5.222	-0.18

1869.83	-5.3849	-5.279	-0.106

1869.92	-5.2362	-5.201	-0.035

1870	-5.2484	-5.135	-0.113

1870.08	-5.1602	-5.095	-0.065

1870.17	-5

1977.71	-4.13	-1.12	8.939	4.692	1.167	4.219	

1977.62	-4.08	-1.11	8.952	4.770	1.159	4.229	

1977.54	-4.18	-1.35	8.888	4.906	1.153	4.152	

1977.46	-4.30	-1.53	8.887	5.071	1.152	4.200	

1977.37	-4.40	-1.26	8.803	5.211	1.140	4.116	

1977.29	-4.57	-1.16	8.743	5.321	1.140	4.086	

1977.21	-4.77	-1.19	8.754	5.422	1.122	4.144	

1977.12	-4.82	-1.17	8.756	5.468	1.086	4.170	

1977.04	-4.81	-1.26	8.751	5.347	1.080	4.027	

1976.96	-4.72	-1.45	8.771	5.258	1.092	4.012	

1976.87	-4.56	-1.22	8.803	5.109	1.120	4.065	

1976.79	-4.36	-0.95	8.894	4.987	1.148	4.119	

1976.71	-4.09	-0.96	8.928	4.893	1.171	4.188	

1976.62	-4.13	-1.10	8.944	4.878	1.188	4.258	

1976.54	-4.37	-1.47	8.923	4.954	1.188	4.256	

1976.46	-4.59	-1.61	8.864	5.108	1.162	4.126	

1976.37	-4.75	-1.51	8.812	5.279	1.130	4.093	

1976.29	-5.02	-1.56	8.739	5.414	1.101	4.041	

1976.21	-5.08	-1.29	8.652	5.489	1.068	4.043	

1976.12	-4.95	-1.31	8.704	5.533	1.017	4.181	

1976.04	-4.81	-1.32	8.699	5.378	1.020	4.605	

1975.96	-4.68	-1.43	8.741	5.392	1.

In [115]:
all_coral_df

,age,d18O,lat,lon,elevation,species


In [62]:
coral_file = 'swart1998-st-1_noaa.txt'
# Example of file with -noaa extension
file1 = open(coral_file,'r')
print(file1.read())
file1.close()

﻿# Gulf of Guinea Coral Isotope, Luminescence and Density Data
#----------------------------------------------------------------------- 
#                World Data Center for Paleoclimatology, Boulder 
#                                  and 
#                     NOAA Paleoclimatology Program 
#----------------------------------------------------------------------- 
# NOTE: Please cite Publication, and Online_Resource and date accessed when using these data. 
# If there is no publication information, please cite Investigators, Title, and Online_Resource and date accessed. 
#
# Online_Resource: https://www.ncdc.noaa.gov/study/1913
#
# Original_Source_URL: http://www1.ncdc.noaa.gov/pub/data/paleo/coral/atlantic/saotome1998.txt
# 
# Description/Documentation lines begin with #
# Data lines have no #
#
# Archive: Corals and Sclerosponges
#--------------------
# Contribution_Date
#	Date: 2005-02-01
#--------------------
# Title
#	Study_Name: Ponta Banana, Principe, Sao Tome - Isotope, Lumi

Gather metadata info! (site lat/lon & years of study)

In [58]:
file1 = open(coral_file,'r')
file_in = file1.readlines()

for line in file_in:
    if "Location:" in line:
        location = line.split(": ")[1]
    if "Northernmost_Latitude:" in line:
        northernmost_lat = line.split(": ")[1]
    if "Southernmost_Latitude:" in line:
        southernmost_lat = line.split(": ")[1]
    if "Easternmost_Longitude:" in line:
        easternmost_lat = line.split(": ")[1]
    if "Westernmost_Longitude:" in line:
        westernmost_lat = line.split(": ")[1]
    if "Elevation:" in line:
        elevation = line.split(": ")[1].split(' ')[0]
    if "Earliest_Year:" in line:
        earliest_year = line.split(": ")[1]
    if "Most_Recent_Year:" in line:
        most_recent_year = line.split(": ")[1]
    if "Species_Name:" in line:
        species = line.split(": ")[1].strip('\n')

lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
lon = (float(easternmost_lat) + float(westernmost_lat)) / 2



In [49]:
age_line = 0
file1 = open(coral_file,'r')
with file1 as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

91
age	d18O	d13C	lumin	dens
 1975	NAN	NAN	-13.274	NAN
 1976	NAN	NAN	1.392	1.664
 1977	-3.60	-1.05	4.481	1.646
 1978	-3.48	-1.17	5.966	1.645
 1979	-3.59	-1.07	7.250	1.686
 1980	-3.69	-1.03	15.983	1.590
 1981	-3.50	-0.87	5.236	1.712
 1982	-3.52	-1.15	10.030	1.675
 1983	-3.66	-1.23	13.752	1.560
 1984	-3.85	-0.89	9.509	1.472
 1985	-3.81	-1.10	5.770	1.483
 1986	-3.51	-1.01	6.514	1.744
 1987	-3.48	-1.12	-1.673	1.712
 1988	-3.27	-0.99	0.708	1.672
 1989	-3.80	-1.39	2.578	1.628
 1990	-3.68	-1.34	0.703	1.727
 1991	-3.28	-1.31	6.420	1.716
 1992	-3.63	-1.04	9.449	1.687
 1993	-3.51	-1.30	NAN	NAN



In [54]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

,age,d18O,d13C,lumin,dens
0,1975,NAN,NAN,-13.274,NAN
1,1976,NAN,NAN,1.392,1.664
2,1977,-3.60,-1.05,4.481,1.646
3,1978,-3.48,-1.17,5.966,1.645
4,1979,-3.59,-1.07,7.250,1.686


Just grab `age` and `d18O` columns! And remove NAN values. 

In [59]:
df_test_d18O = df_test[['age', 'd18O']]
df_test_d18O = df_test_d18O[df_test_d18O['d18O'] != 'NAN']
df_test_d18O['lat'] = lat
df_test_d18O['lon'] = lon
df_test_d18O['elevation'] = elevation
df_test_d18O['species'] = species
df_test_d18O

,age,d18O,lat,lon,elevation,species
2,1977,-3.60,0.47,6.58,-4,Siderastrea spp.
3,1978,-3.48,0.47,6.58,-4,Siderastrea spp.
4,1979,-3.59,0.47,6.58,-4,Siderastrea spp.
5,1980,-3.69,0.47,6.58,-4,Siderastrea spp.
6,1981,-3.50,0.47,6.58,-4,Siderastrea spp.
7,1982,-3.52,0.47,6.58,-4,Siderastrea spp.
8,1983,-3.66,0.47,6.58,-4,Siderastrea spp.
9,1984,-3.85,0.47,6.58,-4,Siderastrea spp.
10,1985,-3.81,0.47,6.58,-4,Siderastrea spp.
11,1986,-3.51,0.47,6.58,-4,Siderastrea spp.
